# Part of Speech Tagging

Several corpora with manual part of speech (POS) tagging are included in NLTK. For this exercise, we'll use a sample of the Penn Treebank corpus, a collection of Wall Street Journal articles. We can access the part-of-speech information for either the Penn Treebank or the Brown as follows. We use sentences here because that is the preferred representation for doing POS tagging.

In [7]:
from nltk.corpus import treebank, brown

print treebank.tagged_sents()[0]
print treebank.sents()[0]
# print brown.tagged_sents()[0]

[(u'Pierre', u'NNP'), (u'Vinken', u'NNP'), (u',', u','), (u'61', u'CD'), (u'years', u'NNS'), (u'old', u'JJ'), (u',', u','), (u'will', u'MD'), (u'join', u'VB'), (u'the', u'DT'), (u'board', u'NN'), (u'as', u'IN'), (u'a', u'DT'), (u'nonexecutive', u'JJ'), (u'director', u'NN'), (u'Nov.', u'NNP'), (u'29', u'CD'), (u'.', u'.')]
[u'Pierre', u'Vinken', u',', u'61', u'years', u'old', u',', u'will', u'join', u'the', u'board', u'as', u'a', u'nonexecutive', u'director', u'Nov.', u'29', u'.']


In NLTK, word/tag pairs are stored as tuples, the transformation from the plain text "word/tag" representation to the python data types is done by the corpus reader.

The two corpus do not have the same tagset; the Brown was tagged with a more fine-grained tagset: for instance, instead of "DT" (determiner) as in the Penn Treebank, the word "the" is tagged as "AT" (article, which is a kind of determiner). We can actually convert them both to the Universal tagset.

In [ ]:
import nltk
nltk.download('universal_tagset')
print treebank.tagged_sents(tagset="universal")[0]
print brown.tagged_sents(tagset="universal")[0]

Now, let's create a basic unigram POS tagger. First, we need to collect POS distributions for each word. We'll do this (somewhat inefficiently) using a dictionary of dictionaries.

In [3]:
from collections import defaultdict

POS_dict = defaultdict(dict)
for word_pos_pair in treebank.tagged_words():
    word = word_pos_pair[0].lower()
    POS = word_pos_pair[1]
    POS_dict[word][POS] = POS_dict[word].get(POS,0) + 1

Let's look at some words which appear with multiple POS, and their POS counts:

In [4]:
for word in POS_dict.keys()[:100]:
    if len(POS_dict[word]) > 1:
        print word
        print POS_dict[word]

increase
{u'VB': 11, u'NN': 20}
refunding
{u'VBG': 1, u'NN': 3}
straight
{u'RB': 2, u'JJ': 3}
second
{u'JJ': 16, u'NNP': 2}
contributed
{u'VBN': 1, u'VBD': 6}
reported
{u'VBN': 12, u'VBD': 25}
elaborate
{u'VB': 7, u'JJ': 2}
climbed
{u'VBN': 1, u'VBD': 4}
reports
{u'VBZ': 2, u'NNS': 13}
sino-u.s.
{u'JJ': 1, u'NNP': 1}
criticism
{u'NN': 1, u'NNP': 1}
brought
{u'VBN': 3, u'VBD': 3}


Common ambiguities that we see here are between nouns and verbs (<i>increase</i>, <i>refunding</i>, <i>reports</i>), and, among verbs, between past tense and past participles (<i>contributed</i>, <i>reported</i>, <i>climbed</i>).

To create an actual tagger, we just need to pick the most common tag for each

In [8]:
tagger_dict = {}
for word in POS_dict:
    tagger_dict[word] = max(POS_dict[word],key=lambda (x): POS_dict[word][x])

def tag(sentence):
    return [(word,tagger_dict.get(word,"NN")) for word in sentence]

print tag(brown.sents()[0])

[(u'The', 'NN'), (u'Fulton', 'NN'), (u'County', 'NN'), (u'Grand', 'NN'), (u'Jury', 'NN'), (u'said', u'VBD'), (u'Friday', 'NN'), (u'an', u'DT'), (u'investigation', u'NN'), (u'of', u'IN'), (u"Atlanta's", 'NN'), (u'recent', u'JJ'), (u'primary', u'JJ'), (u'election', u'NN'), (u'produced', u'VBN'), (u'``', u'``'), (u'no', u'DT'), (u'evidence', u'NN'), (u"''", u"''"), (u'that', u'IN'), (u'any', u'DT'), (u'irregularities', 'NN'), (u'took', u'VBD'), (u'place', u'NN'), (u'.', u'.')]


Though we'd probably want some better handling of capitalized phrases (backing off to NNP rather than NN when a word is capitalized), and there are a few other obvious errors, generally it's not too bad. 

NLTK has built-in support for n-gram taggers; Let's build unigram and bigram taggers, and test their performance. First we need to split our corpus into training and testing

In [12]:
size = int(len(treebank.tagged_sents()) * 0.9)
train_sents = treebank.tagged_sents()[:size]
test_sents = treebank.tagged_sents()[size:]
# print "asdfasdf"


asdfasdf


Let's first compare a unigram and bigram tagger. All NLTK taggers have an evaluate method which prints out the accuracy on some test set.

In [13]:
from nltk import UnigramTagger, BigramTagger

unigram_tagger = UnigramTagger(train_sents)
bigram_tagger = BigramTagger(train_sents)
print unigram_tagger.evaluate(test_sents)
print unigram_tagger.tag(brown.sents()[1])
print bigram_tagger.evaluate(test_sents)
print bigram_tagger.tag(brown.sents()[1])

0.864732824427
[(u'The', u'DT'), (u'jury', u'NN'), (u'further', u'JJ'), (u'said', u'VBD'), (u'in', u'IN'), (u'term-end', None), (u'presentments', None), (u'that', u'IN'), (u'the', u'DT'), (u'City', u'NNP'), (u'Executive', None), (u'Committee', u'NNP'), (u',', u','), (u'which', u'WDT'), (u'had', u'VBD'), (u'over-all', None), (u'charge', u'NN'), (u'of', u'IN'), (u'the', u'DT'), (u'election', u'NN'), (u',', u','), (u'``', u'``'), (u'deserves', None), (u'the', u'DT'), (u'praise', None), (u'and', u'CC'), (u'thanks', u'NNS'), (u'of', u'IN'), (u'the', u'DT'), (u'City', u'NNP'), (u'of', u'IN'), (u'Atlanta', u'NNP'), (u"''", u"''"), (u'for', u'IN'), (u'the', u'DT'), (u'manner', u'NN'), (u'in', u'IN'), (u'which', u'WDT'), (u'the', u'DT'), (u'election', u'NN'), (u'was', u'VBD'), (u'conducted', u'VBN'), (u'.', u'.')]
0.13465648855
[(u'The', u'DT'), (u'jury', u'NN'), (u'further', u'RB'), (u'said', u'VBD'), (u'in', u'IN'), (u'term-end', None), (u'presentments', None), (u'that', None), (u'the', None)

The unigram tagger does way better. The reason is sparsity, the bigram tagger doesn't have counts for many of the word/tag context pairs; what's worse, once it can't tag something, it fails catastrophically for the rest of the sentence tag, because it has no counts at all for missing tag contexts. We can fix this by adding backoffs, including the default tagger with just tags everything as NN

In [15]:
from nltk import DefaultTagger

default_tagger = DefaultTagger("NN")
unigram_tagger = UnigramTagger(train_sents,backoff=default_tagger)
bigram_tagger = BigramTagger(train_sents,backoff=unigram_tagger)

print bigram_tagger.evaluate(test_sents)
print unigram_tagger.evaluate(test_sents)
# print bigram_tagger.tag(brown.sents()[1])

0.891704834606
0.883155216285


We see a 3% increase in performance from adding the bigram information on top of the unigram information.

NLTK has interfaces to the Brill tagger (nltk.tag.brill) and also pre-build, state-of-the-art sequential POS tagging models, for instance the Stanford POS tagger (StanfordPOSTagger), which is what you should use if you actually need high-quality POS tagging for some application; if you are working on a computer with the Stanford CoreNLP tools installed and NLTK set up to use them (this is the case for the lab computers where workshops are held), the below code should work. If not, see the documentation <a href="https://github.com/nltk/nltk/wiki/Installing-Third-Party-Software"> here </a> 

In [18]:
from nltk import StanfordPOSTagger
# print 'asfd'
stanford_tagger = StanfordPOSTagger('english-bidirectional-distsim.tagger')
# print stanford_tagger.tag(brown.sents()[1])

asfd
